In [1]:
import numpy as np
import scipy.sparse as sp
from qecdec import BPDecoder as BPDecoder_Rust
from qecdec import BPDecoder_Py, RotatedSurfaceCode
from qecdec import detector_error_model_to_check_matrices
from time import time


In [2]:
d = 5
p = 0.01
num_shots = 1_000

code = RotatedSurfaceCode(d=d)
circuit = code.make_circuit_memory_z_experiment(
    rounds=d,
    data_qubit_error_rate=p,
    meas_error_rate=p,
    prep_error_rate=p,
    gate1_error_rate=p,
    gate2_error_rate=p,
    keep_z_detectors_only=True
)
detectors_sample, observables_sample = circuit.compile_detector_sampler(seed=0).sample(num_shots, separate_observables=True)
detectors_sample = detectors_sample.astype(np.uint8)
observables_sample = observables_sample.astype(np.uint8)

dem = circuit.detector_error_model()
matrices = detector_error_model_to_check_matrices(dem)
check_matrix, observables_matrix, priors = matrices.check_matrix, matrices.observables_matrix, matrices.priors
check_matrix = check_matrix.toarray().astype(np.uint8)
observables_matrix = observables_matrix.toarray().astype(np.uint8)
priors = priors.astype(np.float64)

In [3]:
decoder_py = BPDecoder_Py(
    H=sp.csr_matrix(check_matrix),
    prior=priors,
    max_iter=50
)

decoder_rust = BPDecoder_Rust(
    pcm=check_matrix,
    prior=priors,
    max_iter=50
)

In [4]:
%timeit decoder_rust.decode(detectors_sample[0])

37.6 μs ± 183 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
%timeit decoder_py.decode(detectors_sample[0])

48.6 ms ± 235 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
t1 = time()
ehat = decoder_rust.decode_batch(detectors_sample)
t2 = time()
print(f"Rust decoder time: {t2 - t1} seconds")

detectors_predict = (ehat @ check_matrix.T) % 2
num_nonconverged = np.sum(np.any(detectors_predict != detectors_sample, axis=1))
print(f"Number of nonconverged: {num_nonconverged}")

observables_predict = (ehat @ observables_matrix.T) % 2
num_logical_errors = np.sum(np.any(observables_predict != observables_sample, axis=1))
print(f"Number of logical errors: {num_logical_errors}")

Rust decoder time: 0.05470085144042969 seconds
Number of nonconverged: 224
Number of logical errors: 150


In [7]:
t1 = time()
ehat = decoder_py.decode_batch(detectors_sample, progress_bar=False)
t2 = time()
print(f"Python decoder time: {t2 - t1} seconds")

detectors_predict = (ehat @ check_matrix.T) % 2
num_nonconverged = np.sum(np.any(detectors_predict != detectors_sample, axis=1))
print(f"Number of nonconverged: {num_nonconverged}")

observables_predict = (ehat @ observables_matrix.T) % 2
num_logical_errors = np.sum(np.any(observables_predict != observables_sample, axis=1))
print(f"Number of logical errors: {num_logical_errors}")

Python decoder time: 70.25258111953735 seconds
Number of nonconverged: 225
Number of logical errors: 136
